In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
import polars as pl
import os
import torch
from torch.utils.data import DataLoader
import numpy as np
import csv

/users/eleves-a/2022/lee.kadz/miniconda3/envs/inf554/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
path_to_data = "../../challenge_data/"

# path_to_training_tweets = path_to_data + "train_tweets/ArgentinaBelgium72.csv"
#path_to_training_tweets = path_to_data + "train_tweets/ArgentinaGermanyFinal77.csv"
path_to_training_tweets = path_to_data + "train_tweets/AustraliaNetherlands29.csv"
path_to_eval_tweets = path_to_data + "eval_tweets"

In [23]:
def load_data(path):
    df = pl.read_csv(path)
    df = df.select(['PeriodID', 'EventType', 'Tweet'])
    return df

df = load_data(path_to_training_tweets)
print(df.head())

shape: (5, 3)
┌──────────┬───────────┬─────────────────────────────────┐
│ PeriodID ┆ EventType ┆ Tweet                           │
│ ---      ┆ ---       ┆ ---                             │
│ i64      ┆ i64       ┆ str                             │
╞══════════╪═══════════╪═════════════════════════════════╡
│ 0        ┆ 0         ┆ RT @FIFAWorldCup: #AUS LINE-UP… │
│ 0        ┆ 0         ┆ “@WSJSports: #USA soccer's Cli… │
│ 0        ┆ 0         ┆ #persieing in his sleep! #AUS … │
│ 0        ┆ 0         ┆ Glad I got my tune in radio to… │
│ 0        ┆ 0         ┆ INSTAGRAM: Are you following t… │
└──────────┴───────────┴─────────────────────────────────┘


In [18]:
## Concaténer les colonnes EventType de plusieurs fichiers CSV

# Liste des fichiers à traiter
path_to_data = "../../challenge_data/train_tweets/"
#files = [path_to_data+"ArgentinaBelgium72.csv", path_to_data+"ArgentinaGermanyFinal77.csv", path_to_data+"AustraliaNetherlands29.csv"]
# files = [path_to_data+"AustraliaNetherlands29.csv", path_to_data+"ArgentinaGermanyFinal77.csv"]
files = [path_to_data+"AustraliaNetherlands29.csv"]


# Liste pour stocker les colonnes EventType de chaque fichier
event_type_data = []

# Charger les colonnes EventType de chaque fichier
for file in files:
    try:
        df = pl.read_csv(file, columns=["EventType"])  # Lire seulement la colonne EventType
        event_type_data.append(df)
    except FileNotFoundError:
        print(f"Erreur : Le fichier {file} est introuvable.")
    except ValueError:
        print(f"Erreur : Le fichier {file} ne contient pas la colonne 'EventType'.")

# Concaténer toutes les données EventType
if event_type_data:
    concatenated_df = pl.concat(event_type_data)

    # Sauvegarder dans un nouveau fichier CSV
    output_file = "ConcatenatedEventTypes.csv"
    concatenated_df.write_csv(output_file)

    print(f"Les 'EventType' ont été concaténés dans {output_file}.")
    # Afficher le nombre de lignes dans le fichier CSV
    with open(output_file, mode="r", newline="") as f:
        reader = csv.reader(f)
        row_count = sum(1 for row in reader) - 1  # Soustraire 1 pour l'en-tête
        print(f"Le fichier '{output_file}' contient {row_count} lignes de données.")
else:
    print("Aucune donnée n'a été concaténée.")

Les 'EventType' ont été concaténés dans ConcatenatedEventTypes.csv.
Le fichier 'ConcatenatedEventTypes.csv' contient 96834 lignes de données.


In [26]:
## Utiliser un modèle d'embedding spécifique pour les tweets

model_name = "vinai/bertweet-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
## Extraire les embeddings [CLS] des tweets pour un match donné

def extract_cls_embeddings_in_batches(model, tokenizer, sentences, batch_size=100, device="cpu"):

    embeddings = []  # Liste pour stocker les embeddings
    dataloader = DataLoader(sentences, batch_size=batch_size, shuffle=False)

    model = model.to(device)  # Déplacer le modèle sur l'appareil spécifié

    for batch in dataloader:
        # Tokenisation des phrases dans le lot courant
        inputs = tokenizer(batch, return_tensors="pt", truncation=True, padding=True).to(device)

        with torch.no_grad():  # Désactiver les calculs de gradient pour économiser de la mémoire
            outputs = model(**inputs)

        # Extraire les embeddings [CLS] du lot courant
        cls_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()  # Revenir sur CPU pour éviter les surcharges
        embeddings.append(cls_embeddings)

    # Concaténer tous les lots en un seul tableau NumPy
    return np.concatenate(embeddings, axis=0)

In [28]:
## Appliquer la fonction extract_cls_embeddings_in_batches()

# Ajouter PeriodID comme premier mot dans Tweet
df_bis = df.with_columns((df['PeriodID'].cast(str) + " " + df['Tweet']).alias('ModifiedTweet'))

# Convertir la colonne modifiée en liste Python
tweets = df_bis['ModifiedTweet'].to_list()

# Extraire les embeddings [CLS] par lots
cls_embeddings = extract_cls_embeddings_in_batches(
    model=model,
    tokenizer=tokenizer,
    sentences=tweets,
    batch_size=100,  # Taille du lot (à ajuster selon votre mémoire)
    device="cuda" if torch.cuda.is_available() else "cpu"  # GPU si disponible
)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [29]:
## Enregistrer les embeddings pour un match dans un fichier CSV

output_file = "cls_embeddings_AustraliaNetherlands29.csv"

# Si le fichier existe déjà, supprimez-le pour éviter les conflits
if os.path.exists(output_file):
    os.remove(output_file)

batch_size = 1000  # Taille du lot pour l'écriture en CSV
header_written = False  # Indicateur pour écrire l'en-tête une seule fois

# Écriture progressive
with open(output_file, mode="w", newline="") as f:
    writer = csv.writer(f)

    for i in range(0, len(cls_embeddings), batch_size):
        batch = cls_embeddings[i:i + batch_size]
        # Convertir en liste de listes si nécessaire
        batch_list = batch.tolist()

        # Écrire l'en-tête (facultatif, selon les dimensions des embeddings)
        if not header_written:
            header = [f"dim_{j}" for j in range(len(batch_list[0]))]
            writer.writerow(header)
            header_written = True

        # Écrire les données
        writer.writerows(batch_list)

print(f"Les embeddings [CLS] ont été enregistrés par lots dans '{output_file}'.")
# Afficher le nombre de lignes dans le fichier CSV
with open(output_file, mode="r", newline="") as f:
    reader = csv.reader(f)
    row_count = sum(1 for row in reader) - 1  # Soustraire 1 pour l'en-tête
    print(f"Le fichier '{output_file}' contient {row_count} lignes de données.")

Les embeddings [CLS] ont été enregistrés par lots dans 'cls_embeddings_AustraliaNetherlands29.csv'.
Le fichier 'cls_embeddings_AustraliaNetherlands29.csv' contient 96834 lignes de données.


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

In [31]:
## Charger les embeddings et les labels de tous les matchs

#files_embedding = ["cls_embeddings_ArgentinaBelgium72.csv", "cls_embeddings_ArgentinaGermanyFinal77.csv", "cls_embeddings_AustraliaNetherlands29.csv"]
# files_embedding = ["cls_embeddings_AustraliaNetherlands29.csv", "cls_embeddings_ArgentinaGermanyFinal77.csv"]
files_embedding = ["cls_embeddings_AustraliaNetherlands29.csv"]


# Charger les embeddings et les labels
def load_embeddings_and_labels(files_embedding,label_file):

    # Charger les labels à partir de ConcatenatedEventTypes.csv
    labels = pl.read_csv(label_file)
    labels = labels.to_numpy().ravel()

    # Convertir les embeddings en numpy array directement à partir de Polars
    embeddings_list = []
    for file in files_embedding:
        embeddings_df = pl.read_csv(file)
        embeddings_list.append(embeddings_df.to_numpy())
    # Concaténer les embeddings (les convertir en numpy array et les empiler)
    embeddings = np.concatenate(embeddings_list, axis=0)

    return embeddings, labels

# Charger les embeddings et les labels
embeddings, labels = load_embeddings_and_labels(files_embedding,"ConcatenatedEventTypes.csv")

# Encoder les labels (EventType) en valeurs numériques
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

In [41]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import accelerate

model_name = "vinai/bertweet-large"

# Charger le modèle avec une tête de classification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Ex. 3 labels pour la classification

# Définir les arguments d'entraînement
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=16,
)

# Préparer les données pour Hugging Face
from datasets import Dataset
dataset = Dataset.from_dict({"text": tweets, "label": labels_encoded.tolist()})

# Tokeniser les données
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Lancer le fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)
trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/bertweet-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/users/eleves-a/2022/lee.kadz/miniconda3/envs/inf554/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

In [34]:
## Diviser les données en ensembles d'entraînement et de validation

# Diviser les données en ensembles d'entraînement et de validation
X_train, X_val, y_train, y_val = train_test_split(embeddings, labels_encoded, test_size=0.3, random_state=42)

# Initialiser le classificateur de régression logistique
model = LogisticRegression(max_iter=10000)

In [35]:
# Entraîner le modèle
model.fit(X_train, y_train)

LogisticRegression(max_iter=10000)

In [36]:
# Faire des prédictions sur l'ensemble de validation
y_pred = model.predict(X_val)

# Calculer la précision
accuracy = accuracy_score(y_val, y_pred)*100
print(f"Validation Accuracy: {accuracy:.2f}%")

Validation Accuracy: 84.18%
